## metro_per_time_merge_processing

### 1. csv merge

* 2020년도 서울시 역별 일별 시간대별 승하차인원 정보(df)
* 2021년도 서울시 역별 일별 시간대별 승하차인원 정보(df2)

In [1]:
import pyspark
import numpy as np
import pandas as pd

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [3]:
#20년도 csv 불러오기
df = spark.read.format("csv").option("header", True).load("./data3/2020_seoul_metro.csv", encoding='euc-kr')
df.show()

+----------+-----+------+----------------------+----+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+
|      날짜| 호선|역번호|                  역명|구분|06:00 이전|06:00 ~ 07:00|07:00 ~ 08:00|08:00 ~ 09:00|09:00 ~ 10:00|10:00 ~ 11:00|11:00 ~ 12:00|12:00 ~ 13:00|13:00 ~ 14:00|14:00 ~ 15:00|15:00 ~ 16:00|16:00 ~ 17:00|17:00 ~ 18:00|18:00 ~ 19:00|19:00 ~ 20:00|20:00 ~ 21:00|21:00 ~ 22:00|22:00 ~ 23:00|23:00 ~ 24:00|24:00 이후|
+----------+-----+------+----------------------+----+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+
|2020-01-01|1호선|   150|        

In [4]:
#데이터타입 확인
df.printSchema()

root
 |-- 날짜: string (nullable = true)
 |-- 호선: string (nullable = true)
 |-- 역번호: string (nullable = true)
 |-- 역명: string (nullable = true)
 |-- 구분: string (nullable = true)
 |-- 06:00 이전: string (nullable = true)
 |-- 06:00 ~ 07:00: string (nullable = true)
 |-- 07:00 ~ 08:00: string (nullable = true)
 |-- 08:00 ~ 09:00: string (nullable = true)
 |-- 09:00 ~ 10:00: string (nullable = true)
 |-- 10:00 ~ 11:00: string (nullable = true)
 |-- 11:00 ~ 12:00: string (nullable = true)
 |-- 12:00 ~ 13:00: string (nullable = true)
 |-- 13:00 ~ 14:00: string (nullable = true)
 |-- 14:00 ~ 15:00: string (nullable = true)
 |-- 15:00 ~ 16:00: string (nullable = true)
 |-- 16:00 ~ 17:00: string (nullable = true)
 |-- 17:00 ~ 18:00: string (nullable = true)
 |-- 18:00 ~ 19:00: string (nullable = true)
 |-- 19:00 ~ 20:00: string (nullable = true)
 |-- 20:00 ~ 21:00: string (nullable = true)
 |-- 21:00 ~ 22:00: string (nullable = true)
 |-- 22:00 ~ 23:00: string (nullable = true)
 |-- 23:00 ~ 24:00:

In [5]:
#21년도 csv에는 23시 이후로 표시되어 있어서 23:00~24:00, 24시 이후 데이터를 합쳐서 표현
df = df.withColumn('23:00 이후', F.col("23:00 ~ 24:00") + F.col("24:00 이후"))
df.show()

#판다스일 때
# pdf["23:00이후"] = pdf["23:00 ~ 24:00"] + pdf["24:00 이후"]
# pdf.drop(["23:00 ~ 24:00", "24:00 이후"], axis=1, inplace=True)
# pdf

+----------+-----+------+----------------------+----+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+
|      날짜| 호선|역번호|                  역명|구분|06:00 이전|06:00 ~ 07:00|07:00 ~ 08:00|08:00 ~ 09:00|09:00 ~ 10:00|10:00 ~ 11:00|11:00 ~ 12:00|12:00 ~ 13:00|13:00 ~ 14:00|14:00 ~ 15:00|15:00 ~ 16:00|16:00 ~ 17:00|17:00 ~ 18:00|18:00 ~ 19:00|19:00 ~ 20:00|20:00 ~ 21:00|21:00 ~ 22:00|22:00 ~ 23:00|23:00 ~ 24:00|24:00 이후|23:00 이후|
+----------+-----+------+----------------------+----+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+


In [6]:
#23:00~24:00와 24시 이후 컬럼 삭제하기.
df = df.drop("23:00 ~ 24:00", "24:00 이후")
df.show()

+----------+-----+------+----------------------+----+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+
|      날짜| 호선|역번호|                  역명|구분|06:00 이전|06:00 ~ 07:00|07:00 ~ 08:00|08:00 ~ 09:00|09:00 ~ 10:00|10:00 ~ 11:00|11:00 ~ 12:00|12:00 ~ 13:00|13:00 ~ 14:00|14:00 ~ 15:00|15:00 ~ 16:00|16:00 ~ 17:00|17:00 ~ 18:00|18:00 ~ 19:00|19:00 ~ 20:00|20:00 ~ 21:00|21:00 ~ 22:00|22:00 ~ 23:00|23:00 이후|
+----------+-----+------+----------------------+----+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+
|2020-01-01|1호선|   150|                서울역|승차|       356|          280|  

In [7]:
#21년도 csv 불러오기
df2 = spark.read.format("csv").option("header", True).load("./data3/서울교통공사_역별 일별 시간대별 승하차인원 정보_20210831.csv", encoding='euc-kr')
df2.show()

+----------+-----+------+----------------------+----+-------+-------+-------+-------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------+
|      날짜| 호선|역번호|                  역명|구분|6시이전|6시-7시|7시-8시|8시-9시|9시-10시|10시-11시|11시-12시|12시-13시|13시-14시|14시-15시|15시-16시|16시-17시|17시-18시|18시-19시|19시-20시|20시-21시|21시-22시|22시-23시|23시이후|
+----------+-----+------+----------------------+----+-------+-------+-------+-------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------+
|2021-01-01|1호선|   150|                서울역|승차|     86|    111|    157|    306|     333|      416|      579|      737|      696|      674|      837|      796|      839|      658|      579|      479|      510|      445|     101|
|2021-01-01|1호선|   150|                서울역|하차|     85|    355|    438|    592|     841|      747|      625|      838|     

In [8]:
df2.printSchema()

root
 |-- 날짜: string (nullable = true)
 |-- 호선: string (nullable = true)
 |-- 역번호: string (nullable = true)
 |-- 역명: string (nullable = true)
 |-- 구분: string (nullable = true)
 |-- 6시이전: string (nullable = true)
 |-- 6시-7시: string (nullable = true)
 |-- 7시-8시: string (nullable = true)
 |-- 8시-9시: string (nullable = true)
 |-- 9시-10시: string (nullable = true)
 |-- 10시-11시: string (nullable = true)
 |-- 11시-12시: string (nullable = true)
 |-- 12시-13시: string (nullable = true)
 |-- 13시-14시: string (nullable = true)
 |-- 14시-15시: string (nullable = true)
 |-- 15시-16시: string (nullable = true)
 |-- 16시-17시: string (nullable = true)
 |-- 17시-18시: string (nullable = true)
 |-- 18시-19시: string (nullable = true)
 |-- 19시-20시: string (nullable = true)
 |-- 20시-21시: string (nullable = true)
 |-- 21시-22시: string (nullable = true)
 |-- 22시-23시: string (nullable = true)
 |-- 23시이후: string (nullable = true)



In [9]:
#20년도 csv파일을 보면 시간별 컬럼명이 다르므로 컬럼명을 기준에맞게 변경해줘야함.
df2 = df2.withColumnRenamed("6시이전", "06:00 이전")\
.withColumnRenamed("6시-7시", "06:00 ~ 07:00")\
.withColumnRenamed("7시-8시", "07:00 ~ 08:00")\
.withColumnRenamed("8시-9시", "08:00 ~ 09:00")\
.withColumnRenamed("9시-10시", "09:00 ~ 10:00")\
.withColumnRenamed("10시-11시", "10:00 ~ 11:00")\
.withColumnRenamed("11시-12시", "11:00 ~ 12:00")\
.withColumnRenamed("12시-13시", "12:00 ~ 13:00")\
.withColumnRenamed("13시-14시", "13:00 ~ 14:00")\
.withColumnRenamed("14시-15시", "14:00 ~ 15:00")\
.withColumnRenamed("15시-16시", "15:00 ~ 16:00")\
.withColumnRenamed("16시-17시", "16:00 ~ 17:00")\
.withColumnRenamed("17시-18시", "17:00 ~ 18:00")\
.withColumnRenamed("18시-19시", "18:00 ~ 19:00")\
.withColumnRenamed("19시-20시", "19:00 ~ 20:00")\
.withColumnRenamed("20시-21시", "20:00 ~ 21:00")\
.withColumnRenamed("21시-22시", "21:00 ~ 22:00")\
.withColumnRenamed("22시-23시", "22:00 ~ 23:00")\
.withColumnRenamed("23시이후", "23:00 이후")

df2.show()

+----------+-----+------+----------------------+----+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+
|      날짜| 호선|역번호|                  역명|구분|06:00 이전|06:00 ~ 07:00|07:00 ~ 08:00|08:00 ~ 09:00|09:00 ~ 10:00|10:00 ~ 11:00|11:00 ~ 12:00|12:00 ~ 13:00|13:00 ~ 14:00|14:00 ~ 15:00|15:00 ~ 16:00|16:00 ~ 17:00|17:00 ~ 18:00|18:00 ~ 19:00|19:00 ~ 20:00|20:00 ~ 21:00|21:00 ~ 22:00|22:00 ~ 23:00|23:00 이후|
+----------+-----+------+----------------------+----+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+
|2021-01-01|1호선|   150|                서울역|승차|        86|          111|  

In [10]:
#df1, df2 csv 파일 합치기
timely_df = df.union(df2)
timely_df

DataFrame[날짜: string, 호선: string, 역번호: string, 역명: string, 구분: string, 06:00 이전: string, 06:00 ~ 07:00: string, 07:00 ~ 08:00: string, 08:00 ~ 09:00: string, 09:00 ~ 10:00: string, 10:00 ~ 11:00: string, 11:00 ~ 12:00: string, 12:00 ~ 13:00: string, 13:00 ~ 14:00: string, 14:00 ~ 15:00: string, 15:00 ~ 16:00: string, 16:00 ~ 17:00: string, 17:00 ~ 18:00: string, 18:00 ~ 19:00: string, 19:00 ~ 20:00: string, 20:00 ~ 21:00: string, 21:00 ~ 22:00: string, 22:00 ~ 23:00: string, 23:00 이후: string]

In [11]:
df.count()

202280

In [12]:
df2.count()

135876

In [13]:
timely_df.count()

338156

In [14]:
#잘못된 데이터가 있는지 확인
timely_df.select("역명").distinct().show(30)

+------------------+
|              역명|
+------------------+
|            한양대|
|              혜화|
|              금호|
|              신답|
|         을지로3가|
|              미사|
|              약수|
|충정로(경기대입구)|
|              명동|
|          버티고개|
|  안암(고대병원앞)|
|              반포|
|    천호(풍납토성)|
|            당고개|
|          김포공항|
|        영등포구청|
|            낙성대|
|              지축|
|            보라매|
|          강동구청|
|            장한평|
|    부천종합운동장|
|            서대문|
|봉화산(서울의료원)|
|            사가정|
|              남성|
|              신길|
|            돌곶이|
|              내방|
|            신중동|
+------------------+
only showing top 30 rows



In [16]:
#null값 확인
timely_df.where(F.col('23:00 이후').isNull()==True).count()

0

In [17]:
#이상값 확인
timely_df.where(F.col('06:00 이전')<0).count()

0

In [22]:
#합친 데이터 csv파일로 저장
timely_df.toPandas().to_parquet("time_per_metro.parquet")

In [25]:
timely_df.show()

+----------+-----+------+----------------------+----+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+
|      날짜| 호선|역번호|                  역명|구분|06:00 이전|06:00 ~ 07:00|07:00 ~ 08:00|08:00 ~ 09:00|09:00 ~ 10:00|10:00 ~ 11:00|11:00 ~ 12:00|12:00 ~ 13:00|13:00 ~ 14:00|14:00 ~ 15:00|15:00 ~ 16:00|16:00 ~ 17:00|17:00 ~ 18:00|18:00 ~ 19:00|19:00 ~ 20:00|20:00 ~ 21:00|21:00 ~ 22:00|22:00 ~ 23:00|23:00 이후|
+----------+-----+------+----------------------+----+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+
|2020-01-01|1호선|   150|                서울역|승차|       356|          280|  

In [ ]:
#필요없는 컬럼 제거
timely_df = timely_df.drop("호선", "역번호")
timely_df.show()

In [ ]:
#컬럼 데이터타입 변환
df = df.withColumn('날짜',F.to_date(F.col("날짜"),"yyyy-MM-dd"))\
.withColumn('06:00 이전',F.col('06:00 이전').cast(T.IntegerType()))\
.withColumn('06:00 ~ 07:00',F.col('06:00 ~ 07:00').cast(T.IntegerType()))\
.withColumn('07:00 ~ 08:00',F.col('07:00 ~ 08:00').cast(T.IntegerType()))\
.withColumn('08:00 ~ 09:00',F.col('08:00 ~ 09:00').cast(T.IntegerType()))\
.withColumn('09:00 ~ 10:00',F.col('09:00 ~ 10:00').cast(T.IntegerType()))\
.withColumn('10:00 ~ 11:00',F.col('10:00 ~ 11:00').cast(T.IntegerType()))\
.withColumn('11:00 ~ 12:00',F.col('11:00 ~ 12:00').cast(T.IntegerType()))\
.withColumn('12:00 ~ 13:00',F.col('12:00 ~ 13:00').cast(T.IntegerType()))\
.withColumn('13:00 ~ 14:00',F.col('13:00 ~ 14:00').cast(T.IntegerType()))\
.withColumn('14:00 ~ 15:00',F.col('14:00 ~ 15:00').cast(T.IntegerType()))\
.withColumn('15:00 ~ 16:00',F.col('15:00 ~ 16:00').cast(T.IntegerType()))\
.withColumn('16:00 ~ 17:00',F.col('16:00 ~ 17:00').cast(T.IntegerType()))\
.withColumn('17:00 ~ 18:00',F.col('17:00 ~ 18:00').cast(T.IntegerType()))\
.withColumn('18:00 ~ 19:00',F.col('18:00 ~ 19:00').cast(T.IntegerType()))\
.withColumn('19:00 ~ 20:00',F.col('19:00 ~ 20:00').cast(T.IntegerType()))\
.withColumn('20:00 ~ 21:00',F.col('20:00 ~ 21:00').cast(T.IntegerType()))\
.withColumn('21:00 ~ 22:00',F.col('21:00 ~ 22:00').cast(T.IntegerType()))\
.withColumn('22:00 ~ 23:00',F.col('22:00 ~ 23:00').cast(T.IntegerType()))\
.withColumn('23:00 ~ 24:00',F.col('23:00 ~ 24:00').cast(T.IntegerType()))\
.withColumn('24:00 이후',F.col('24:00 이후').cast(T.IntegerType()))

In [ ]:
#Null 값 확인
df.where(col('06:00 이전'))